<a href="https://colab.research.google.com/github/adidror005/youtube-videos/blob/main/TradingStream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

### Install Python SDK


In [20]:
!pip install alpaca-py

### Grab API key and Secret

In [21]:
from google.colab import userdata
ALPACA_API_KEY = userdata.get('ALPACA_API_KEY')
ALPACA_API_SECRET = userdata.get('ALPACA_API_SECRET')

### Connect to Trading Client

In [22]:
from alpaca.trading.client import TradingClient
trading_client = TradingClient(ALPACA_API_KEY, ALPACA_API_SECRET, paper=True)

### Nest_Asnycio
* Needed for running asyncio in Jupyter Credit Ewald De Wit!

In [24]:
import nest_asyncio
nest_asyncio.apply()

# Trading Stream

### Where we can handle events related to orders we placed.
For example,
* New Order
* Replaced Order
* Filled Order

### Initialize Trading Stream

In [25]:
from alpaca.trading.stream import TradingStream
trading_stream = TradingStream(ALPACA_API_KEY, ALPACA_API_SECRET)

### Example 1: Very Basic Example of TradingStream Couroutine

In [28]:
??TradeUpdate
??TradeEvent

In [29]:
from alpaca.trading.models import TradeUpdate
from alpaca.trading.enums import OrderStatus, OrderSide,TradeEvent
from alpaca.trading.requests import LimitOrderRequest
from alpaca.trading.enums import TimeInForce
import numpy as np

trading_stream = TradingStream(ALPACA_API_KEY, ALPACA_API_SECRET)

async def on_order_update(trade_update:TradeUpdate):
  order_id = trade_update.order.id
  side = trade_update.order.side
  event = trade_update.event
  price = trade_update.price
  qty = trade_update.qty
  timestamp = trade_update.timestamp
  print(f"OrderId: {order_id}")
  print(f"Timestamp: {timestamp}")
  print(f"Side: {side}")
  print(f"Event: {event}")
  print(f"Price: {price}")
  print(f"Qty: {qty}")



### Subscribe to "Trade" Updates
**Note** I wish they could rename it because this can be confusing from market data trade updates...

In [30]:
trading_stream.subscribe_trade_updates(on_order_update)

### Run the Websocket Stream
* This is blocking later we can try the "non-blocking" version.

In [31]:
trading_stream.run()

OrderId: 2732ad45-433b-49fc-b6fd-ffee1f932044
Timestamp: 2025-05-16 11:21:58.683150+00:00
Side: OrderSide.BUY
Event: canceled
Price: None
Qty: None
OrderId: a480f13d-f447-4bb8-abd9-d604ae2b83e7
Timestamp: 2025-05-16 11:21:58.686700+00:00
Side: OrderSide.BUY
Event: canceled
Price: None
Qty: None
OrderId: 2e36e34f-735c-4d75-943e-c5f0da41d809
Timestamp: 2025-05-16 11:23:05.298404+00:00
Side: OrderSide.BUY
Event: pending_new
Price: None
Qty: None
OrderId: 2e36e34f-735c-4d75-943e-c5f0da41d809
Timestamp: 2025-05-16 11:23:05.301936+00:00
Side: OrderSide.BUY
Event: new
Price: None
Qty: None
OrderId: 56f978ce-baac-4927-9d03-e41c34fae102
Timestamp: 2025-05-16 11:23:37.685036+00:00
Side: OrderSide.BUY
Event: pending_new
Price: None
Qty: None
OrderId: 56f978ce-baac-4927-9d03-e41c34fae102
Timestamp: 2025-05-16 11:23:37.690378+00:00
Side: OrderSide.BUY
Event: new
Price: None
Qty: None
OrderId: 2a43f40f-d6fe-4d8b-8c06-d22f8b39185a
Timestamp: 2025-05-16 11:24:10.155066+00:00
Side: OrderSide.BUY
Event:

### Let's trigger a take profit 1% Higher after a buy fill

In [33]:
from alpaca.trading.models import TradeUpdate
from alpaca.trading.enums import OrderStatus, OrderSide,TradeEvent
from alpaca.trading.requests import LimitOrderRequest
from alpaca.trading.enums import TimeInForce
import numpy as np

trading_stream = TradingStream(ALPACA_API_KEY, ALPACA_API_SECRET)

async def on_order_update(trade_update:TradeUpdate):
  order_id = trade_update.order.id
  symbol = trade_update.order.symbol
  side = trade_update.order.side
  event = trade_update.event
  price = trade_update.price
  qty = trade_update.qty
  timestamp = trade_update.timestamp
  print(f"OrderId: {order_id}")
  print(f"Symbol: {symbol}")
  print(f"Timestamp: {timestamp}")
  print(f"Side: {side}")
  print(f"Event: {event}")
  print(f"Price: {price}")
  print(f"Qty: {qty}")
  if event in [TradeEvent.PARTIAL_FILL,TradeEvent.FILL]:
    take_profit_price = np.round(price * 1.01,2)
    take_profit_qty = qty
    take_profit_side = OrderSide.SELL
    limit_order_request = LimitOrderRequest(
        symbol=symbol,
        qty=take_profit_qty,
        side=take_profit_side,
        limit_price = take_profit_price,
        extended_hours=True,
        time_in_force=TimeInForce.DAY
    )
    take_profit_order = trading_client.submit_order(
        order_data=limit_order_request
    )
    print(f"Trying to Submit Take Profit Order for {take_profit_qty} shares at {take_profit_price}")



### Subsribe to Trade Updates and Run

In [ ]:
trading_stream.subscribe_trade_updates(on_order_update)
trading_stream.run()

OrderId: 2e36e34f-735c-4d75-943e-c5f0da41d809
Symbol: AMD
Timestamp: 2025-05-16 11:34:22.687141+00:00
Side: OrderSide.BUY
Event: canceled
Price: None
Qty: None
OrderId: 56f978ce-baac-4927-9d03-e41c34fae102
Symbol: AMD
Timestamp: 2025-05-16 11:34:22.702451+00:00
Side: OrderSide.BUY
Event: canceled
Price: None
Qty: None
OrderId: 2f9d0e3b-e41f-4970-a5b8-f07928d764c6
Symbol: AMD
Timestamp: 2025-05-16 11:35:04.572745+00:00
Side: OrderSide.BUY
Event: pending_new
Price: None
Qty: None
OrderId: 2f9d0e3b-e41f-4970-a5b8-f07928d764c6
Symbol: AMD
Timestamp: 2025-05-16 11:35:04.576062+00:00
Side: OrderSide.BUY
Event: new
Price: None
Qty: None
OrderId: 2f9d0e3b-e41f-4970-a5b8-f07928d764c6
Symbol: AMD
Timestamp: 2025-05-16 11:35:04.712641+00:00
Side: OrderSide.BUY
Event: partial_fill
Price: 118.09
Qty: 1.0
Trying to Submit Take Profit Order for 1.0 shares at 119.27
OrderId: 2f9d0e3b-e41f-4970-a5b8-f07928d764c6
Symbol: AMD
Timestamp: 2025-05-16 11:35:04.871949+00:00
Side: OrderSide.BUY
Event: partial_